In [2]:
import numpy as np
import pandas as pd

In [3]:
# set maximum columns to display
pd.set_option("display.max_columns",50)

In [3]:
# load weather data
weather = pd.read_csv('../Data/Raw/weather.csv', low_memory=False)

In [4]:
# load electricity load data
load = pd.read_excel('../Data/Raw/native_Load_2017.xlsx')

In [5]:
# WEATHER DATA CLEANUP

In [6]:
# remove daily and monthly stats from hourly temperature readings
weather = weather.iloc[:,:26]

In [7]:
# subset weather to necessary hourly columns
weather = weather.iloc[:,[1,5,7,9,11,13,15,16,17,20,24]]

In [8]:
# in hourly precipication column, remove 's' from some readings
# replace 'T' (trace amount) with 0.001 since minimum reported value is 0.01
# convert column to numeric

In [9]:
weather.HOURLYPrecip = weather.HOURLYPrecip.str.replace('s', '')

In [10]:
weather.HOURLYPrecip = weather.HOURLYPrecip.str.replace('T', '0.001')

In [11]:
weather.HOURLYPrecip = weather.HOURLYPrecip.astype('float')

In [12]:
# convert other columns to numbers

In [13]:
weather.HOURLYDRYBULBTEMPC = weather.HOURLYDRYBULBTEMPC.str.replace('s', '')

In [14]:
weather.HOURLYDRYBULBTEMPC = weather.HOURLYDRYBULBTEMPC.astype('float')

In [15]:
weather.HOURLYDewPointTempC = weather.HOURLYDewPointTempC.str.replace('s', '')

In [16]:
weather.HOURLYDewPointTempC = weather.HOURLYDewPointTempC.astype('float')

In [17]:
weather.HOURLYWindSpeed = weather.HOURLYWindSpeed.str.replace('s', '').astype('float')

In [18]:
weather.HOURLYStationPressure = weather.HOURLYStationPressure.str.replace('s', '').astype('float')

In [19]:
# add column for region name

In [20]:
def applyRegions(station):
    stations = list(weather.STATION_NAME.unique())
    regions = list(load.columns[1:-1])
    regions = [regions[1], regions[2], regions[7], regions[3], regions[5], regions[6], regions[0], regions[4]]
    stations_regions = dict(zip(stations, regions))
    return stations_regions[station]

In [21]:
# this line takes a while to run
weather['REGION'] = weather.apply(lambda row: applyRegions(row['STATION_NAME']), axis=1)

In [22]:
# drop station name

In [23]:
weather = weather.drop('STATION_NAME', axis=1)

In [24]:
# convert date column to datetime format

In [25]:
weather.DATE = pd.to_datetime(weather.DATE)

In [26]:
# set multi-index

In [27]:
weather = weather.set_index(['DATE', 'REGION'])

In [28]:
# unstack regions

In [29]:
weather = weather.unstack()

In [30]:
# round to to hour

In [31]:
weather = weather.resample('H', level=0).mean()

In [32]:
# shift forward by 1 hour

In [33]:
weather.index = weather.index.shift(1)

In [34]:
# save weather dataframe

In [35]:
weather.to_csv('../Data/Cleaned/weather.csv')

In [36]:
# LOAD DATA CLEANUP

In [37]:
# drop ERCOT total column

In [38]:
load = load.drop('ERCOT', axis=1)

In [39]:
# generate datetime column for hour

In [40]:
load['DATE'] = pd.date_range('2017-01-01 01:00:00', '2018-01-01 00:00:00', freq='H')

In [41]:
# drop hour ending column and set new index

In [42]:
load = load.drop(['Hour Ending'], axis=1)

In [43]:
load = load.set_index('DATE')

In [44]:
# save load dataframe

In [45]:
load.to_csv('../Data/Cleaned/load.csv')

In [46]:
# LOAD CLEANED DATAFRAMES

In [47]:
weather = pd.read_csv('../Data/Cleaned/weather.csv', header=[0,1], index_col=0)

In [48]:
load = pd.read_csv('../Data/Cleaned/load.csv', index_col=0)

In [49]:
# label column axes

In [50]:
weather = weather.rename_axis(['INDICATOR', 'REGION'], axis=1)

In [51]:
# drop unneeded indicators

In [52]:
weather = weather.drop(['HOURLYDewPointTempC', 'HOURLYStationPressure'], axis=1, level=0)

In [53]:
# join data frames

In [54]:
weather = weather.stack()

In [55]:
load = pd.DataFrame(load.stack())

In [56]:
load.columns = ['LOAD']

In [57]:
load.index.names = ['DATE', 'REGION']

In [58]:
df = weather.join(load)

In [59]:
# rename columns

In [60]:
df.columns

Index(['HOURLYDRYBULBTEMPC', 'HOURLYPrecip', 'HOURLYRelativeHumidity',
       'HOURLYWETBULBTEMPC', 'HOURLYWindSpeed', 'LOAD'],
      dtype='object')

In [61]:
df.columns = ['DryBulbTempC', 'Precip', 'Humidity','WetBulbTempC', 'WindSpeed', 'Load']

In [62]:
# unstack

In [63]:
df = df.unstack()

In [64]:
# reorder column index levels

In [65]:
df.columns.names = ['INDICATOR', 'REGION']

In [66]:
df = df.reorder_levels(['REGION','INDICATOR'], axis=1)

In [67]:
# reorder columns

In [68]:
df = df.groupby(level=[0,1], axis=1).mean()

In [69]:
df.head()

REGION                     COAST                                         \
INDICATOR           DryBulbTempC Precip Humidity WetBulbTempC WindSpeed   
DATE                                                                      
2017-01-01 01:00:00         18.9    0.0     90.0    17.900000       0.0   
2017-01-01 02:00:00         18.9    0.0     93.0    18.200000       0.0   
2017-01-01 03:00:00         18.3    0.0     97.0    18.000000       3.0   
2017-01-01 04:00:00         18.5    0.0     92.0    17.766667       2.0   
2017-01-01 05:00:00         18.3    0.0     93.0    17.700000       0.0   

REGION                                   EAST                               \
INDICATOR                   Load DryBulbTempC Precip Humidity WetBulbTempC   
DATE                                                                         
2017-01-01 01:00:00  8791.789509         15.0    0.0    100.0         15.0   
2017-01-01 02:00:00  8569.708419         15.0    0.0    100.0         15.0   
2017-01-01 03:00:00  8326.425638         15.0    0.0    100.0         15.0   
2017-01-01 04:00:00  8137.497400         15.0    0.0    100.0         15.0   
2017-01-01 05:00:00  8011.869581         15.0    0.0    100.0         15.0   

REGION                                           FWEST                  \
INDICATOR           WindSpeed        Load DryBulbTempC Precip Humidity   
DATE                                                                     
2017-01-01 01:00:00       5.0  896.746302          9.4  0.001     69.0   
2017-01-01 02:00:00       5.0  865.930568          9.4  0.000     69.0   
2017-01-01 03:00:00       6.5  839.051175          8.9  0.000     74.0   
2017-01-01 04:00:00       7.0  822.829332          7.2  0.000     80.0   
2017-01-01 05:00:00       4.5  814.016188          8.3  0.000     77.0   

REGION                                                         NCENT         \
INDICATOR           WetBulbTempC WindSpeed         Load DryBulbTempC Precip   
DATE                                                                          
2017-01-01 01:00:00          6.8       6.0  1997.717635          8.9    0.0   
2017-01-01 02:00:00          6.8       6.0  1997.781319          8.3    0.0   
2017-01-01 03:00:00          6.7       8.0  1993.699160          7.8    0.0   
2017-01-01 04:00:00          5.6       3.0  1995.540876          7.2    0.0   
2017-01-01 05:00:00          6.5       7.0  1995.253501          6.1    0.0   

REGION                                                                  NORTH  \
INDICATOR           Humidity WetBulbTempC WindSpeed         Load DryBulbTempC   
DATE                                                                            
2017-01-01 01:00:00     63.0          5.9       6.0  9239.153285          4.4   
2017-01-01 02:00:00     71.0          6.0       8.0  9104.997245          2.8   
2017-01-01 03:00:00     71.0          5.5       6.0  8988.035201          1.7   
2017-01-01 04:00:00     74.0          5.2       6.0  8979.148462          3.3   
2017-01-01 05:00:00     80.0          4.6       8.0  9033.547636          3.9   

REGION                                                                  \
INDICATOR           Precip Humidity WetBulbTempC WindSpeed        Load   
DATE                                                                     
2017-01-01 01:00:00    0.0     65.0          2.0       9.0  683.621986   
2017-01-01 02:00:00    0.0     73.0          1.0       9.0  677.969375   
2017-01-01 03:00:00    0.0     78.0          0.4       8.0  671.998949   
2017-01-01 04:00:00    0.0     73.0          1.5       7.0  675.267971   
2017-01-01 05:00:00    0.0     70.0          1.9       8.0  663.619875   

REGION                     SCENT                                           \
INDICATOR           DryBulbTempC Precip   Humidity WetBulbTempC WindSpeed   
DATE                                                                        
2017-01-01 01:00:00    11.850000    0.0  98.000000       11.800      0.00  

In [70]:
# save combined df

In [71]:
df.to_csv('../Data/Cleaned/combined.csv')

In [3]:
# further preprocessing

In [4]:
df = pd.read_csv('../Data/Cleaned/combined.csv', header=[0,1], index_col=0)

In [ ]:
# keep only temperature and load

In [5]:
df = df.loc[:, (slice(None), ['DryBulbTempC', 'Load'])]

In [40]:
# calculate square of temperature

In [14]:
tempSq = df.loc[:, (slice(None), 'DryBulbTempC')]**2

In [ ]:
# swap levels to prepare for join

In [22]:
tempSq = tempSq.swaplevel(axis=1)

In [24]:
df = df.swaplevel(axis=1)

In [ ]:
# join square of temperature

In [28]:
df = df.join(tempSq, rsuffix='Sq')

In [ ]:
# swap levels back and sort

In [30]:
df = df.swaplevel(axis=1)

In [33]:
df = df.sort_index(axis=1)

In [34]:
df.head()

REGION                     COAST                                     EAST  \
INDICATOR           DryBulbTempC DryBulbTempCSq         Load DryBulbTempC   
DATE                                                                        
2017-01-01 01:00:00         18.9         357.21  8791.789509         15.0   
2017-01-01 02:00:00         18.9         357.21  8569.708419         15.0   
2017-01-01 03:00:00         18.3         334.89  8326.425638         15.0   
2017-01-01 04:00:00         18.5         342.25  8137.497400         15.0   
2017-01-01 05:00:00         18.3         334.89  8011.869581         15.0   

REGION                                                FWEST                 \
INDICATOR           DryBulbTempCSq        Load DryBulbTempC DryBulbTempCSq   
DATE                                                                         
2017-01-01 01:00:00          225.0  896.746302          9.4          88.36   
2017-01-01 02:00:00          225.0  865.930568          9.4          88.36   
2017-01-01 03:00:00          225.0  839.051175          8.9          79.21   
2017-01-01 04:00:00          225.0  822.829332          7.2          51.84   
2017-01-01 05:00:00          225.0  814.016188          8.3          68.89   

REGION                                  NCENT                              \
INDICATOR                   Load DryBulbTempC DryBulbTempCSq         Load   
DATE                                                                        
2017-01-01 01:00:00  1997.717635          8.9          79.21  9239.153285   
2017-01-01 02:00:00  1997.781319          8.3          68.89  9104.997245   
2017-01-01 03:00:00  1993.699160          7.8          60.84  8988.035201   
2017-01-01 04:00:00  1995.540876          7.2          51.84  8979.148462   
2017-01-01 05:00:00  1995.253501          6.1          37.21  9033.547636   

REGION                     NORTH                                   SCENT  \
INDICATOR           DryBulbTempC DryBulbTempCSq        Load DryBulbTempC   
DATE                                                                       
2017-01-01 01:00:00          4.4          19.36  683.621986    11.850000   
2017-01-01 02:00:00          2.8           7.84  677.969375    10.975000   
2017-01-01 03:00:00          1.7           2.89  671.998949    13.300000   
2017-01-01 04:00:00          3.3          10.89  675.267971    13.925000   
2017-01-01 05:00:00          3.9          15.21  663.619875    13.933333   

REGION                                                 SOUTH                 \
INDICATOR           DryBulbTempCSq         Load DryBulbTempC DryBulbTempCSq   
DATE                                                                          
2017-01-01 01:00:00     140.422500  4490.781365         16.1         259.21   
2017-01-01 02:00:00     120.450625  4370.656830         15.6         243.36   
2017-01-01 03:00:00     176.890000  4210.650003         15.6         243.36   
2017-01-01 04:00:00     193.905625  4088.713039         16.1         259.21   
2017-01-01 05:00:00     194.137778  4021.757095         16.7         278.89   

REGION                                   WEST                             
INDICATOR                   Load DryBulbTempC DryBulbTempCSq        Load  
DATE                                                                      
2017-01-01 01:00:00  2366.632745     5.900000      34.810000  954.192864  
2017-01-01 02:00:00  2332.744630     6.033333      36.401111  951.025166  
2017-01-01 03:00:00  2237.506202     7.000000      49.000000  944.357749  
2017-01-01 04:00:00  2178.102265     6.583333      43.340278  943.188703  
2017-01-01 05:00:00  2133.953870     5.733333      32.871111  954.937932

In [35]:
# save pre-processed dataframe

In [36]:
df.to_csv('../Data/Cleaned/preprocessed.csv')